Conf Video URL - https://www.youtube.com/watch?v=MjHpMCIvwsY

### Example 1: Timing
How long does it take for a funciton to run?

In [26]:
import time

def logtime(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        total_time = time.time() - start_time

        with open('timelog.txt', 'w') as outfile:
            outfile.write(f'{time.time()}\t{func.__name__}\t{total_time}')
        return result
    return wrapper

if __name__ == '__main__':
    @logtime
    def slow_add(a, b):
        time.sleep(2)
        return a + b

    @logtime
    def slow_mul(a, b):
        time.sleep(3)
        return a * b

    for i in range(5):
        for j in range(5):
            print(slow_add(i, j))
            print(slow_mul(i, j))

0
0
1
0
2
0
3
0
4
0
1
0
2
1
3
2
4
3
5
4
2
0
3
2
4
4
5
6
6
8
3
0
4
3
5
6
6
9
7
12
4
0
5
4
6
8
7
12
8
16


### Example 2: Once per minute
Raise an exeception if we try to run a function more than once in 60 seconds

In [24]:
import time

class CalledTooOftenError(Exception): 
    pass

def once_per_minute(func):
    last_invoked = 0

    def wrapper(*args, **kwargs):
        nonlocal last_invoked
        elapsed_time = time.time() - last_invoked

        if elapsed_time < 60:
            raise CalledTooOftenError(f'Only {elapsed_time} has passed')
        last_invoked = time.time()
        return func(*args, **kwargs)

    return wrapper

if __name__ == '__main__':
    @once_per_minute
    def add(a, b):
        return a + b

    print(add(2, 2))
    print(add(3, 3))


4


CalledTooOftenError: Only 0.0009996891021728516 has passed

### Example 3: Once per n
Raise an exeception if we try to run a function more than once in n seconds

In [22]:
import time

class CalledTooOftenError(Exception): 
    pass

def once_per_n(n):
    def middle(func):
        last_invoked = 0

        def wrapper(*args, **kwargs):
            nonlocal last_invoked
            elapsed_time = time.time() - last_invoked

            if elapsed_time < n:
                raise CalledTooOftenError(f'Only {elapsed_time} has passed')
            last_invoked = time.time()
            return func(*args, **kwargs)
        return wrapper
    return middle

if __name__ == '__main__':
    @once_per_n(5)
    def slow_add(a, b):
        time.sleep(3)
        return a + b

        
    print(slow_add(2, 2))
    print(slow_add(3, 3))
    print(slow_add(4, 4))


4


CalledTooOftenError: Only 3.0127012729644775 has passed

### Example 4: Memoization
Cache the result of function calls, so we don't need to call them again

In [25]:
import pickle

def memoize(func):
    cache = {}

    def wrapper(*args, **kwargs):
        t = (pickle.dumps(args), pickle.dumps(kwargs))

        if t not in cache:
            print(f'Caching NEW value for {func.__name__}{args}')
            cache[t] = func(*args, **kwargs)
        else:
            print(f'Using OLD value for {func.__name__}{args}')

        return cache[t]
    return wrapper


if __name__ == '__main__':
    @memoize
    def add(a, b):
        print("Running add!")
        return a + b

    @memoize
    def mul(a, b):
        print("Running mul!")
        return a * b

    @memoize
    def mysum(numbers, **kwargs):
        print(f"Running mysum, message = '{kwargs.get('message')}'")
        total = 0
        for one_number in numbers:
            total += one_number
        return total

    print(add(3, 7))
    print(mul(3, 7))
    print(add(3, 7))
    print(mul(3, 7))
    print(mysum([1, 2, 3, 4, 5], message='hello'))
    print(mysum([1, 2, 3, 4, 5], message='hello'))
    print(mysum([1, 2, 3, 4, 5], message='goodbye'))

Caching NEW value for add(3, 7)
Running add!
10
Caching NEW value for mul(3, 7)
Running mul!
21
Using OLD value for add(3, 7)
10
Using OLD value for mul(3, 7)
21
Caching NEW value for mysum([1, 2, 3, 4, 5],)
Running mysum, message = 'hello'
15
Using OLD value for mysum([1, 2, 3, 4, 5],)
15
Caching NEW value for mysum([1, 2, 3, 4, 5],)
Running mysum, message = 'goodbye'
15


### Example 5: Attributes
Give many objects the same attributes, but without using inheritance

In [20]:
import time

def fancy_repr(self):
    return f'I am a {type(self).__name__}, with vars {vars(self)}'

def repr_and_birthyday(c):
    c.__repr__ = fancy_repr

    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        o._created_at = time.time()
        return o
    return wrapper

if __name__=='__main__':
    @repr_and_birthyday
    class Foo():
        def __init__(self, x, y):
            self.x = x
            self.y = y
    
    f = Foo(10, [10, 20, 30])
    print(f)
    print(f._created_at)

I am a Foo, with vars {'x': 10, 'y': [10, 20, 30], '_created_at': 1636986571.6216035}
1636986571.6216035


In [1]:
import time

def object_birthday(c):
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        o._created_at = time.time()
        return o
    return wrapper

@object_birthday
class Foo:
    def __init__(self, x, y):
        self.x = x
        self.y = y

f = Foo(10, [10,20,30])
print(f)
print(f._created_at)

1636988095.9131477


In [2]:
def fancy_repr(self):
    return f"I'm a {type(self).__name__}, with vars {vars(self)}"

def better_repr(c):
    c.__repr__ = fancy_repr
    return c

@better_repr
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y


f = Foo(10, [10, 20, 30])
print(f)

I'm a Foo, with vars {'x': 10, 'y': [10, 20, 30]}
